## _**To understand leftanti join and in sql statement**_

In [0]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
from pyspark.sql.functions import lit

lst_candidates = [('1', 'Alice'),('2', 'Bob'),('3', 'Charlie'),('4', 'Diana'),('5', 'Eve')]
lst_blacklist =[('2','aa'),('4','xx'),lit(None)]

df1_schema_lst = StructType([StructField("id", StringType(), True),
                             StructField("name", StringType(), True)])
df2_schema_lst = StructType([StructField("id", StringType(),True),
                             StructField("name", StringType(), True)])

dfl_candidates = spark.createDataFrame(lst_candidates, df1_schema_lst )
dfl_blacklist = spark.createDataFrame(lst_blacklist,df2_schema_lst )

dfl_candidates.printSchema()
dfl_blacklist.printSchema()

dfl_candidates.createOrReplaceTempView("candidates")
dfl_blacklist.createOrReplaceTempView("blacklist")

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)



In [0]:
%sql
select * from candidates

id,name
1,Alice
2,Bob
3,Charlie
4,Diana
5,Eve


In [0]:
%sql
select * from blacklist

id,name
2,aa
4,xx
null,null


In [0]:
dfl_candidates.join(dfl_blacklist,"id",how="leftanti").show()

+---+-------+
| id|   name|
+---+-------+
|  1|  Alice|
|  3|Charlie|
|  5|    Eve|
+---+-------+



In [0]:
%sql
select * from candidates where id not in(select id from blacklist --where coalesce(id,"-1") != '-1'
)

id,name


In [0]:
%sql
select c.* from candidates c
left anti join blacklist b on b.id = c.id

id,name
1,Alice
3,Charlie
5,Eve
